反卷积网络，通过测量输出和已知输入重构未知输入的过程。在神经网络中，反卷积过程并不具备学习的能力，仅仅是**用于可视化**一个已经训练好的卷积网络模型，没有学习训练的过程。

下图为VGG 16反卷积神经网络结构，展示了一个卷积网络与反卷积网络结合的过程。VGG 16是一个深度神经网络模型。它的反卷积就是将中间的数据，按照前面卷积、池化等变化的过程，完全相反地做一遍，从而得到类似原始输入的数据。
![](imgs/19_vgg16.png)

## 反卷积神经网络应用场景
由于反卷积网络的特性，导致它有许多特别的应用，一般可以用于**信道均衡、图像恢复、语音识别、地震学、无损探伤**等未知输入估计和过程辨别方面的问题。

在神经网络的研究中，反卷积更多是冲淡可视化的作用。对于一个复杂的深度卷积网络，通过每层若干个卷积核的变换，我们无法知道每个卷积核关注的是什么，变换后的特征是什么样子。通过**反卷积的还原**，可以对这些问题有个清晰的可视化，以各层得到的特征图作为输入，进行反卷积，得到反卷积结果，用以验证显示各层提取到的特征图。

## 反卷积原理
反卷积，可以理解为卷积操作的逆操作。这里千万不要当成反卷积操作可以复原卷积操作的输入值，反卷积并没有这个功能，它仅仅是将卷积变换过程中的步骤反向变换一个而已，通过将卷积核转置，与卷积后的结果再做一遍卷积，所以它还有个名字叫**转置卷积**。
虽然它不能还原出原来卷积的样子，但是在作用上具有类似的效果，可以将带有小部分缺失的信息最大化地恢复，也可以用来恢复被卷积生成后的原始输入。

反卷积具体操作比较复杂，具体步骤如下：
1. 首先是将卷积核反转（并不是转置，而是上下左右方向进行递序操作）。
2. 再将卷积结果作为输入，做补0的扩充操作，即往每一个元素后面补0.这一步是根据步长来的，对每一个元素沿着步长的方向补（步长-1）个0.例如，步长为1就不用补0了。
3. 在扩充后的输入基础上再对整体补0.以原始输入的shape作为输出，按照前面卷积padding规则，计算padding的补0位置及个数，得到的补0位置要上下和左右各自颠倒一下。
4. 将补0后的卷积结果作为真正的输入，翻转后的卷积核为filter，进行步长为1的卷积操作。

> 计算padding按规则补0 时，统一按照padding='SAME'、步长为1x1的方式来计算。


以一个[1,4,4,1]矩阵为例，进行filter为2x2，步长为2x2的卷积操作，反卷积操作如下图：![](imgs/19_deconvolution.png)

在反卷积过程中，首先将2x2矩阵通过步长补0的方式变成4x4，再通过padding反方向补0，然后与反转后的filter使用步长为1x1的卷积操作，最终得出结果。但是这个结果已经与原来的全1矩阵不等了，说明转置卷积只能回复部分特征，无法百分百地恢复原始数据。

## 演示反卷积操作
在TensorFlow中反卷积是通过函数tf.nn.conv2d_transpose来实现的，定义如下：
___
```python
conv2d_transpose(value,
                filter,
                output_shape,
                strides,
                padding='SAME',
                data_format='NHWC',
                name=None)
```
___
- value:代表通过卷积操作之后的张量，一般用NHWC类型。
- filter:代表卷积核。
- output_shape:代表输出的张量形状也是个四维张量。
- padding：代表原数据生成value时用的补0的方式，是用来检查输入形状和输出形状是否合规的。
- strides:步长。
- return : 反卷积后的结果，按照output_shape指定的形状。

> NHWC类型是神经网络中在处理图像方面常用的类型，4个字母分别代表4个意思，即N-个数、H-高、W-宽度、C-通道数。也就是常见的四维张量。

> output_shape并**不是一个随便写的形状**，它必须是能够生成value参数的元数据的形状，如果输出形状不对，函数会报错。
反卷积操作其实是使用了gen_nn_ops.conv2d_backprop_input函数来最终实现的，相当于TensorFlow中利用了卷积操作在反向传播的处理函数中做反卷积操作，即卷积操作的反向传播就是反卷积操作。

### 实例描述
通过模拟数据进行卷积与反卷积的操作，来比较卷积与反卷积中padding在SAME、VALID下的变化。

In [2]:
import numpy as np
import tensorflow as tf

#模拟数据
img = tf.Variable(tf.constant(1.0,shape=[1,4,4,1]))

filter = tf.Variable(tf.constant([1.0,0,-1,-2],shape=[2,2,1,1]))

#分别进行VALID 和 SAME操作
conv = tf.nn.conv2d(img,filter=filter,strides=[1,2,2,1],padding='VALID')
cons = tf.nn.conv2d(img,filter,strides=[1,2,2,1],padding='SAME')
print(conv.shape)
print(cons.shape)

#进行反卷积
contv = tf.nn.conv2d_transpose(conv,filter,[1,4,4,1],strides=[1,2,2,1],padding='VALID')
conts = tf.nn.conv2d_transpose(cons,filter,[1,4,4,1],strides=[1,2,2,1],padding='SAME')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print("conv:\n",sess.run([conv,filter]))
    print("cons:\n",sess.run([cons]))
    print("contv:\n",sess.run([contv]))
    print("conts:\n",sess.run([conts]))    

(1, 2, 2, 1)
(1, 2, 2, 1)
conv:
 [array([[[[-2.],
         [-2.]],

        [[-2.],
         [-2.]]]], dtype=float32), array([[[[ 1.]],

        [[ 0.]]],


       [[[-1.]],

        [[-2.]]]], dtype=float32)]
cons:
 [array([[[[-2.],
         [-2.]],

        [[-2.],
         [-2.]]]], dtype=float32)]
contv:
 [array([[[[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]],

        [[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]]]], dtype=float32)]
conts:
 [array([[[[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]],

        [[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]]]], dtype=float32)]
